In [8]:
import csv
import pandas as pd
import numpy as np
from patsy import dmatrices
from pulp import *
import matplotlib.pyplot as plt


from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_poisson_deviance
from sklearn.model_selection import cross_val_score

import seaborn as sns
from sklearn.cluster import KMeans

import statsmodels.api as sm

calcio_A = pd.read_pickle("calcio_A.pk1")
calcio_C = pd.read_pickle("calcio_C.pk1")
calcio_D = pd.read_pickle("calcio_D.pk1")
calcio = pd.read_pickle("calcio.pk1")

# Part 3: Pulp Optimization

Now we have assigned each player to a cluster, we can begin looking into using the PuLP optimizer to determine which combination of player tiers will get us the highest points total.

First, we need to wrangle the data some more. We start by dropping players with value below 0 to avoid confusing the algorithm. (We won't miss them as they are technically the worst buys)

There are many columns we don't need any more and these are dropped using the pop method. We also reindex the dataframe to avoid any indexing errors during processing the optimization algorithm.

In [9]:
calcio = calcio.loc[calcio["21/22_Value"] > 0]

dict = calcio.set_index("Nome").T.to_dict("index")
calcio = calcio.reset_index(drop=True)

keys_to_remove = ["Id", "Squadra", "Pg", "Mv", "Mf", "Gf", "Gs", "Rp", "Rc", "R+", "R-", "Ass", "Asf", "Amm", "Esp", "Unnamed: 18",
                 "IV","Au", "FV", "D", "BID", "ln(Bid)", "FV 19/20", "scaled_bid", "y_hat", "z_hat", "20/21_Value", "residual", "cluster"]

for i in keys_to_remove:
    
    dict.pop(i)

    calcio = calcio.drop(columns = i)
    

calcio = calcio.dropna()

newindex = []
for i in range(calcio.shape[0]):
    newindex.append(i)
    

calcio = pd.merge(pd.DataFrame(newindex), calcio, how='left', left_index = True, right_index = True)

pd.set_option('display.max_rows', 1000)

print(calcio)


       0  index  R              Nome  FV 20/21  2 YR AVG  21/22_Value Tier
0      0     16  A           BERARDI      8.37  8.196608   197.522789  Hig
1      1      1  A            LUKAKU      8.89  8.703271   261.110957  Hig
2      2      3  A         ZAPATA D.      7.88  8.139945   190.411280  Hig
3      3      0  A           RONALDO      8.79  8.969938   294.578695  Hig
4      4      8  A            CAPUTO      7.65  7.869946   156.525444  Hig
5      5      2  A          IMMOBILE      7.70  8.296612   210.073667  Hig
6      6     15  A            MURIEL      8.92  8.606605   248.978907  Hig
7      7      4  A       IBRAHIMOVIC      8.28  8.243275   203.379698  Hig
8      8     30  A          LAPADULA      6.50  6.756621    16.798818  Low
9      9     18  A              BOGA      6.46  6.709955    10.942007  Low
10    10     21  A       RAFAEL LEAO      6.78  6.779953    19.727027  Low
11    11     23  A          ORSOLINI      6.74  6.723286    12.615179  Low
12    12     13  A       

Above we can see our simplified dataframe that is ready to be optimized. 

There is just one more thing to do which is one-hot encode the "R" column which is the position of the player (A, C or D). We will see later why this is important but it is in relation the way one of the constraints is written for the optimization.

In [10]:
dummies = pd.get_dummies(calcio["R"])


calcio = pd.merge(calcio, dummies, how='right', left_index = True, right_index = True)



Now our data is ready for the optimization. We create the pulp problem, calling it the "PointsMaximizer" and we define the problem as a maximization problem. Now we have to create our decision variables, and for each player we append the Name, position and tier. In the line below, we define each value as binary: they can only be either 1 or 0- they can only be in the team or not.

In [11]:
prob = pulp.LpProblem('PointsMaximizer', LpMaximize)

decision_variables = []

for rownum, row in calcio.iterrows():
    variable = str('x' + str(row["Nome"])+str(row["R"])+str(row["Tier"]))
    variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
    decision_variables.append(variable)
    
print ("Total number of decision_variables: " + str(len(decision_variables)))
print ("Array with Decision Variables:" + str(decision_variables))

Total number of decision_variables: 117
Array with Decision Variables:[xBERARDIAHig, xLUKAKUAHig, xZAPATA_D.AHig, xRONALDOAHig, xCAPUTOAHig, xIMMOBILEAHig, xMURIELAHig, xIBRAHIMOVICAHig, xLAPADULAALow, xBOGAALow, xRAFAEL_LEAOALow, xORSOLINIALow, xJOAO_PEDROAMid, xMERTENSAMid, xBARROWAMid, xMARTINEZ_L.AMid, xREBICAMid, xQUAGLIARELLAAMid, xSANCHEZAMid, xCORREAAMid, xCAICEDOAMid, xINSIGNEAMid, xBELOTTIAMid, xLOZANOAMid, xDZEKOAMid, xPETAGNAAMid, xPANDEVAMid, xJANKTOCMid, xRUIZCMid, xTRAORE'_HJ.CMid, xSENSICMid, xLOCATELLICMid, xBARELLACMid, xZACCAGNICMid, xBONAVENTURACMid, xKUCKACMid, xKULUSEVSKICMid, xLAZOVICCMid, xPELLEGRINI_LO.CMid, xCASTROVILLICMid, xLAZZARICMid, xPOLITANOCMid, xNANDEZCMid, xBROZOVICCMid, xERIKSENCMid, xDJURICICCMid, xNAINGGOLANCMid, xKESSIE'CHig, xLUIS_ALBERTOCHig, xSORIANOCHig, xMALINOVSKYICHig, xZIELINSKICHig, xMKHITARYANCHig, xDE_PAULCHig, xVERETOUTCHig, xPASALICCHig, xCALHANOGLUCHig, xMILINKOVIC_SAVICCHig, xCHIESACHig, xVERDICLow, xSAELEMAEKERSCLow, xRABIOTCLow, 

Now we define the total_points formula, which is what we will be optimizing. We iterate over the df and decision variable, and when the name matches up we multiply the decision variable by their 2 Year average. 

This creates the optimization formula, which then gets added to the PuLP problem.

In [12]:
total_points = ""



for i, schedule in enumerate(decision_variables):
    for rownum, row in calcio.iterrows():
        if rownum  == i:
                   
                      
            formula = row["2 YR AVG"]*schedule
                 
            total_points = lpSum([total_points, formula])
        
            
            
prob += total_points
print ("Optimization function: " + str(total_points))


Optimization function: 6.10329159*xACERBIDMid + 6.17662311*xALEX_SANDRODMid + 6.39328896*xANSALDIDMid + 6.43662057*xBARELLACMid + 6.93328584*xBARROWAMid + 7.53661353*xBELOTTIAMid + 6.01662513*xBENTANCURCLow + 8.19660825*xBERARDIAHig + 5.68329225*xBERESZYNSKIDLow + 6.16329099*xBIRAGHIDMid + 6.70995453*xBOGAALow + 6.40662246*xBONAVENTURACMid + 6.2399568*xBONUCCIDMid + 6.333288*xBREMERDMid + 6.44662194*xBROZOVICCMid + 6.11662491*xBRUNO_PERESDMid + 7.09328298*xCAICEDOAMid + 5.76995973*xCALDARADLow + 6.97995192*xCALHANOGLUCHig + 7.86994623*xCAPUTOAHig + 6.14995833*xCASTILLEJOCLow + 6.43662189*xCASTROVILLICMid + 6.23329164*xCHIELLINIDMid + 7.33661475*xCHIESACHig + 6.98328489*xCORREAAMid + 6.25329054*xCRISCITODMid + 6.48662034*xCUADRADODMid + 6.93995034*xD'AMBROSIODHig + 6.25662195*xDANILODMid + 6.40328715*xDARMIANDMid + 6.36995679*xDEMIRALDMid + 6.32662254*xDE_LIGTDMid + 7.27328148*xDE_PAULCHig + 6.3866226*xDE_VRIJDMid + 6.15662361*xDI_LORENZODMid + 6.08995773*xDJIMSITIDMid + 6.5799543000000

Now we write a similar formula but this time we multiply the decision variable by the cost of the player. However we aren't trying to maximize this formula- this will be one our constraints. We want to keep the cumulative budget lower than the budget we set.

I decided to only optimize the 10 starting positions, excluding the goal keeper. We set 100 of the 1000 credit budget aside for the keepers, and 100 aside for bench and reserve outfield players. These budgets can be adjusted but for now I settled with a budget of 700 for the outfield starting players.

We add the constraint to the PuLP problem; that the total cumulative budget must be less or equal to than our set budget.

In [13]:
budget = 800
totalbudget = ""


for rownum, row in calcio.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            formula = row["21/22_Value"]*schedule
            totalbudget = lpSum([totalbudget, formula])
            

                
            
prob += (totalbudget <= budget)     

Now we write our final constraints. These are in order to make the players chosen fit into the decided starting formation. If we weren't to do this we could end up with 10 Attackers, which is not a viable formation in the fantacalcio game.

We iterate over the schedule again but this is where the one hot encoded position columns become useful. E.g row["A"] would give a 0 for every player that is not an attacker and they would not be included in the Attacking Player constraint.

We set ourselves a formation of 4 3 3, and add these constraints to the problem. There is an argument for having the optimizer pick us a formation, but for now we just settled for the renowned 4 3 3, as the consensus is this is the strongest way to set up your squad, having 4 defenders in order to get the defenders bonus mentioned earlier, and 3 attackers getting the bulk of the goals.

In [14]:
Aplayers = ""
for rownum, row in calcio.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            formula = row["A"]*schedule
            Aplayers = formula + Aplayers

Cplayers = ""
for rownum, row in calcio.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            formula = row["C"]*schedule
            Cplayers = formula + Cplayers
            
Dplayers = ""
for rownum, row in calcio.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            formula = row["D"]*schedule
            Dplayers = formula + Dplayers
            



prob += (Aplayers == 3)
prob += (Cplayers == 3)
prob += (Dplayers == 4)    

Now we can print our problem, and write it to an LP file. We can look at all our constraints below and make sure they all look correct according to what we are trying to achieve.

In [15]:
print(prob)
prob.writeLP("PointsMaximizer.lp")

PointsMaximizer:
MAXIMIZE
6.10329159*xACERBIDMid + 6.17662311*xALEX_SANDRODMid + 6.39328896*xANSALDIDMid + 6.43662057*xBARELLACMid + 6.93328584*xBARROWAMid + 7.53661353*xBELOTTIAMid + 6.01662513*xBENTANCURCLow + 8.19660825*xBERARDIAHig + 5.68329225*xBERESZYNSKIDLow + 6.16329099*xBIRAGHIDMid + 6.70995453*xBOGAALow + 6.40662246*xBONAVENTURACMid + 6.2399568*xBONUCCIDMid + 6.333288*xBREMERDMid + 6.44662194*xBROZOVICCMid + 6.11662491*xBRUNO_PERESDMid + 7.09328298*xCAICEDOAMid + 5.76995973*xCALDARADLow + 6.97995192*xCALHANOGLUCHig + 7.86994623*xCAPUTOAHig + 6.14995833*xCASTILLEJOCLow + 6.43662189*xCASTROVILLICMid + 6.23329164*xCHIELLINIDMid + 7.33661475*xCHIESACHig + 6.98328489*xCORREAAMid + 6.25329054*xCRISCITODMid + 6.48662034*xCUADRADODMid + 6.93995034*xD'AMBROSIODHig + 6.25662195*xDANILODMid + 6.40328715*xDARMIANDMid + 6.36995679*xDEMIRALDMid + 6.32662254*xDE_LIGTDMid + 7.27328148*xDE_PAULCHig + 6.3866226*xDE_VRIJDMid + 6.15662361*xDI_LORENZODMid + 6.08995773*xDJIMSITIDMid + 6.5799543000

[xACERBIDMid,
 xALEX_SANDRODMid,
 xANSALDIDMid,
 xBARELLACMid,
 xBARROWAMid,
 xBELOTTIAMid,
 xBENTANCURCLow,
 xBERARDIAHig,
 xBERESZYNSKIDLow,
 xBIRAGHIDMid,
 xBOGAALow,
 xBONAVENTURACMid,
 xBONUCCIDMid,
 xBREMERDMid,
 xBROZOVICCMid,
 xBRUNO_PERESDMid,
 xCAICEDOAMid,
 xCALDARADLow,
 xCALHANOGLUCHig,
 xCAPUTOAHig,
 xCASTILLEJOCLow,
 xCASTROVILLICMid,
 xCHIELLINIDMid,
 xCHIESACHig,
 xCORREAAMid,
 xCRISCITODMid,
 xCUADRADODMid,
 xD'AMBROSIODHig,
 xDANILODMid,
 xDARMIANDMid,
 xDEMIRALDMid,
 xDE_LIGTDMid,
 xDE_PAULCHig,
 xDE_VRIJDMid,
 xDI_LORENZODMid,
 xDJIMSITIDMid,
 xDJURICICCMid,
 xDZEKOAMid,
 xERIKSENCMid,
 xFARAONIDMid,
 xGAGLIOLODLow,
 xGODINDLow,
 xGOSENSDHig,
 xHATEBOERDMid,
 xHERNANDEZ_T.DHig,
 xHYSAJDLow,
 xIBRAHIMOVICAHig,
 xIMMOBILEAHig,
 xINSIGNEAMid,
 xIZZODLow,
 xJANKTOCMid,
 xJOAO_PEDROAMid,
 xKESSIE'CHig,
 xKJAERDMid,
 xKOLAROVDLow,
 xKOULIBALYDLow,
 xKUCKACMid,
 xKULUSEVSKICMid,
 xKUMBULLADLow,
 xLAPADULAALow,
 xLAZOVICCMid,
 xLAZZARICMid,
 xLOCATELLICMid,
 xLOZANOAMid,
 

Now we are at our final stage. We will solve the problem and note each solution, both the amounts of players from each tier, and the budgets too.

We iterate over the variables of the solution and clean up so we are just left with the Name of the player. This is then used to compute the budget of each A, C and D positions. After that we make variables for each tier in each position, and add that up.

We are left with with the spread of players across each tier and how much budget was allocated to each position. However running this once isn't enough- we will run it numerous times and take an average solution.

In [16]:
optimization_result = prob.solve()


print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", value(prob.objective))
print("Individual decision_variables: ")


players = []
players2 = []
Dbudget=0
Cbudget=0
Abudget=0

for i in calcio["Nome"]:

    i = i.replace(" ", "_")
    


for v in prob.variables():
    if v.varValue == 1.0:
        players.append(str(v.name)[-4:])
        players2.append(str(v.name[1:-4]))
     
                
        
for i in players2:
        
   
    if calcio.loc[calcio["Nome"] == i.replace("_", " ")].R.max() == "D":

        Dbudget += calcio["21/22_Value"].loc[calcio["Nome"] == i.replace("_", " ")].max()

    elif calcio.loc[calcio["Nome"] == i.replace("_", " ")].R.max() == "C":

        Cbudget += calcio["21/22_Value"].loc[calcio["Nome"] == i.replace("_", " ")].max()

    elif calcio.loc[calcio["Nome"] == i.replace("_", " ")].R.max() == "A":

        Abudget += calcio["21/22_Value"].loc[calcio["Nome"] == i.replace("_", " ")].max()
        
        
AHig = 0
AMid = 0
ALow = 0
CHig = 0
CMid = 0
CLow = 0
DHig = 0
DMid = 0
DLow = 0



for i in players:
    
    if i == "AHig":
        AHig += 1
    elif i == "AMid":
        AMid += 1
    elif i == "ALow":
        ALow += 1
        
    elif i == "CHig":
        CHig += 1
    elif i == "CMid":
        CMid += 1
    elif i == "CLow":
        CLow += 1

    elif i == "DHig":
        DHig += 1
    elif i == "DMid":
        DMid += 1
    elif i == "DLow":
        DLow += 1
        
print("Striker High: " + str(AHig))
print("Striker Mid: " + str(AMid))
print("Striker Low: " + str(ALow))

print("Midfielder High: " + str(CHig))
print("Midfieler Mid: " + str(CMid))
print("Midfielder Low " + str(CLow))

print("Defender High: " + str(DHig))
print("Defender Mid: " + str(DMid))
print("Defender Low " + str(DLow))

print(Dbudget)
print(Cbudget)
print(Abudget)

Status: Optimal
Optimal Solution to the problem:  71.90282385
Individual decision_variables: 
Striker High: 0
Striker Mid: 2
Striker Low: 1
Midfielder High: 3
Midfieler Mid: 0
Midfielder Low 0
Defender High: 3
Defender Mid: 1
Defender Low 0
135.1437838048692
291.5875904660068
272.9541150408917


We want to run this code numerous times so we would like to log each solution into a csv file that can later be analyzed. We will write a list including the allocations in each tier, and budget used for each position. 

However, we will need a .py file in order to run from the command line. The file will essentially be the same as this one, but without using the print function as we no longer need visual represantation, and commenting the printing out will enable the code to run faster. The code used to run the program 1000 times from the command line is located in the "runscript.txt" file

In [18]:
with open("budget.csv", "a", newline="") as csvfile:
    writer = csv.writer(csvfile)
    list = [str(AHig), str(AMid), str(ALow), str(CHig), str(CMid), str(CLow), str(DHig), str(DMid), str(DLow), str(Abudget), str(Cbudget), str(Dbudget)]
    writer.writerow(list)

# Conclusion & Improvements

After running the code 850 times (technical difficulty), we have plenty of data to inspect. If we load up the "budget.csv" we can analyze these optimizations. 
Looking at the mode of each budget allocation, we can see that the most popular way to split the budget is the following:

- Striker High: 1
- Striker Mid: 2
- Striker Low: 0
- Midfielder High: 3
- Midfielder Mid: 0
- Midfielder Low 0
- Defender High: 3
- Defender Mid: 1
- Defender Low 0

We take the average of the budget allocation too:

- Strikers: 387.0734682
- Midfielders: 257.8340112
- Defenders: 154.7775558

Now we have an outline of the most economical way to spend the budget based on the types on players that are available, and it doesn't matter who the players are. This is definetly useful to us and we did infact spend our budget in a way close to this, and ended up with a good team which is currently third in the league, although our one "High" priced striker is currently injured.


This was certainly a good excercise in terms of optimization and I look forward to using the PuLP library more extensively in future. I will likely make some tweaks to this ahead of next season's auction and there are a few fields I am interested in improving.

Firstly, I would like to explore the regression some more. Perhaps implementing a new statistical model or improving the current linear model with an exponential term for the higher end players, as this is where it seemed to struggle. More reliable player price predictions will allow us to do more advanced analysis, and also enable us to snipe deals in the live auction.

Another area to improve would be to introduce more metrics for each player. Currently, we only use the data from the official site; leghe.fantacalcio.it, which gives us goals, assists, bookings and the all important player ratings. However, this doesn't include more interesting stats like expected goals (xG) and assists (xA), where we can see a player who has been previously unlucky, and might be undervalued based on what he could do. Also taking notes of minutes played and injuries would be important to gauge a players value, as a player who is often unavaialble or not selected, would be less valuable as one who averages the same rating but over more games.